In [1]:
%pip install langchain
%pip install langchain_groq
%pip install langchain_core
%pip install langchain_community

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
### Model Environments
import os
from dotenv import load_dotenv
load_dotenv()


# Groq API Key if you want to use Groq models
from langchain_groq import ChatGroq
groq_api_key = os.getenv("GROQ_API_KEY")
groqModelName = 'llama-3.3-70b-versatile'
groqModel = ChatGroq(model=groqModelName,groq_api_key=groq_api_key)

model = groqModel
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000016D9A0F5CC0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000016D9A0F7070>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [74]:
from langchain_core.messages import HumanMessage, AIMessage

messages = [
    HumanMessage("Hello, how are you?"),
    AIMessage("I am fine, how are you?"),
    HumanMessage("I am fine too, thank you!"),
]

model.invoke(messages)

AIMessage(content="It's great to hear that you're doing well. Is there something I can help you with or would you like to chat for a bit?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 67, 'total_tokens': 97, 'completion_time': 0.109439098, 'prompt_time': 0.007073221, 'queue_time': 0.341424525, 'total_time': 0.116512319}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_5f849c5a0b', 'finish_reason': 'stop', 'logprobs': None}, id='run-8bf08a4a-2890-4414-b06e-dd68de38da9f-0', usage_metadata={'input_tokens': 67, 'output_tokens': 30, 'total_tokens': 97})

In [12]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory(session_id=session_id)
    return store[session_id]

message_history = RunnableWithMessageHistory(
    model,
    get_session_history
)

In [ ]:
from uuid import uuid4

def generate_session_id():
    return str(uuid4())

def getConfig(session_id):
    config = {
        "configurable": {
            "session_id": session_id,
        }
    }
    return config

5b5468e0-7bed-4d4d-ac71-0f154e08f861


In [ ]:
spesific_session_id = 'chat1'
response = message_history.invoke(messages, getConfig(spesific_session_id))

In [22]:
response.content

"That's great to hear. Is there something I can help you with or would you like to chat?"

In [23]:
new_messages = [
    HumanMessage("how am i today?"),
]
response = message_history.invoke(new_messages, getConfig('chat1'))

In [24]:
response.content

"You said you're fine, so that's a good start. You also seem to be in a friendly and conversational mood, which is great. Would you like to talk about anything in particular or just see where the conversation goes?"

In [26]:
print(store)

{'chat_1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}), AIMessage(content='I am fine, how are you?', additional_kwargs={}, response_metadata={}), HumanMessage(content='I am fine too, thank you!', additional_kwargs={}, response_metadata={}), AIMessage(content="That's great to hear. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 67, 'total_tokens': 89, 'completion_time': 0.08, 'prompt_time': 0.005812967, 'queue_time': 0.12043747799999999, 'total_time': 0.085812967}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_f07de61dcd', 'finish_reason': 'stop', 'logprobs': None}, id='run-601f1d03-12a8-4166-a052-3f506ff5d1f0-0', usage_metadata={'input_tokens': 67, 'output_tokens': 22, 'total_tokens': 89}), HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_met

In [68]:
### prompt templates way
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("user", "Hello, how are you?"),
        ("system", "I am fine, how are you?"),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [69]:
new_config = getConfig('chat2')
response = chain.invoke({"messages": messages}, new_config)
response.content

"It's great to hear that you're doing well. Is there anything I can help you with or would you like to chat for a bit? I'm here to listen and assist you with any questions or topics you'd like to discuss."

In [71]:
message_history = RunnableWithMessageHistory(
    chain,
    get_session_history
)
new_messages = [
    HumanMessage("how am i today?"),
]
new_config = getConfig('chat2')
response = message_history.invoke(new_messages, new_config)

In [72]:
response.content

'You told me earlier that you\'re "fine", which suggests that you\'re doing okay, but maybe not feeling particularly excited or energetic. You also mentioned that you\'re learning Python and LangChain, which can be a bit challenging, but you\'re willing to ask questions and learn, which is great! So, taking all that into account, it seems like you\'re having a fairly normal day, with a mix of learning and maybe a bit of curiosity. Is that about right?'

In [77]:
### prompt templates a little bit more complex
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant and you should answer only {language} language"),
        ("user", "Hello, how are you?"),
        ("system", "I am fine, how are you?"),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

new_config = getConfig('chat4')
response = message_history.invoke({"messages": messages,"language":"Turkish"}, new_config)
response.content

'İyiyim, teşekkür ederim! Gününüz nasıl geçiyor?'

In [ ]:
### history management with trimmer

from langchain_core.messages import trim_messages
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

max_tokens = 45
trimmerStrategy = "last"
token_counter = model
system_message_included = True
allow_partial_answers = False


trimmer = trim_messages(
    max_tokens=max_tokens,
    strategy=trimmerStrategy,
    token_counter=token_counter,
    include_system=system_message_included,
    allow_partial=allow_partial_answers,
    start_on="human",
)

messages = [
    SystemMessage(content="You are a helpful assistant"),
    HumanMessage(content="Hello, I am Cihan"),
    AIMessage(content="Hello, I am a language model Cihan"),
    HumanMessage(content="How are you?"),
    AIMessage(content="I am fine, Do you like ice cream?"),
    HumanMessage(
        content="Yes, I like chocolate ice cream, vanilla ice cream, strawberry ice cream, pistachio ice cream, and many more"
    ),
    AIMessage(content="I like ice cream too"),
    HumanMessage(content="What is your favorite color?"),
    AIMessage(content="I like purple"),
    HumanMessage(content="I like blue"),
    AIMessage(content="I like blue too"),
    # HumanMessage(content="So, what is your favorite food?"),
]

In [89]:
trimmedMessages = trimmer.invoke(messages)
trimmedMessages

[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is your favorite color?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='I like purple', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like blue', additional_kwargs={}, response_metadata={}),
 AIMessage(content='I like blue too', additional_kwargs={}, response_metadata={})]

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain =(
    RunnablePassthrough(messages=itemgetter("messages") | trimmer) | prompt | model
)

messages.append(HumanMessage(content="What is my favorite color?"))

response = chain.invoke({"messages": messages, "language": "Turkish"})
response.content

'Mavi, daha önce söylemiştin'

In [100]:
%pip install chromadb
%pip install langchain_chroma
%pip install langchain_core
%pip install langchain_huggingface

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [112]:
## with vector store
from langchain_core.documents import Document

documents = [
    Document(
        page_content="I like chocolate ice cream, vanilla ice cream, strawberry ice cream, pistachio ice cream, and many more",
        metadata={"author": "Cihan"},
    ),
    Document(
        page_content="I like blue",
        metadata={"author": "Cihan"},
    ),
    Document(
        page_content="I like purple",
        metadata={"author": "Cihan"},
    ),
]


In [113]:
## Embedding
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [114]:
## vector store
from langchain_chroma import Chroma
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
chroma = Chroma.from_documents(documents, embeddings)
vectorStore = chroma
vectorStore

In [115]:
vectorStore.similarity_search("blue")

[Document(id='f0bcea62-04c5-45f6-9041-bcb68ab14193', metadata={'author': 'Cihan'}, page_content='I like blue'),
 Document(id='5d21a64c-2581-4dab-b213-49507333231f', metadata={'author': 'Cihan'}, page_content='I like blue'),
 Document(id='83bb873f-4905-41b8-9294-1d119326a7a5', metadata={'author': 'Cihan'}, page_content='I like blue'),
 Document(id='69b72bb6-13c2-42ca-9ddd-d98eba3e8292', metadata={'author': 'Cihan'}, page_content='I like blue')]

In [117]:
### async query
await vectorStore.asimilarity_search("blue")

[Document(id='f0bcea62-04c5-45f6-9041-bcb68ab14193', metadata={'author': 'Cihan'}, page_content='I like blue'),
 Document(id='5d21a64c-2581-4dab-b213-49507333231f', metadata={'author': 'Cihan'}, page_content='I like blue'),
 Document(id='83bb873f-4905-41b8-9294-1d119326a7a5', metadata={'author': 'Cihan'}, page_content='I like blue'),
 Document(id='69b72bb6-13c2-42ca-9ddd-d98eba3e8292', metadata={'author': 'Cihan'}, page_content='I like blue')]

In [118]:
## retriever
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(
    vectorStore.similarity_search).bind(k=1)
retriever.batch(['blue','vanilla'])

[[Document(id='83bb873f-4905-41b8-9294-1d119326a7a5', metadata={'author': 'Cihan'}, page_content='I like blue')],
 [Document(id='324478cc-bbca-439f-9d3b-600a6174d5b5', metadata={'author': 'Cihan'}, page_content='I like chocolate ice cream, vanilla ice cream, strawberry ice cream, pistachio ice cream, and many more')]]

In [119]:
retriever = vectorStore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)
retriever.batch(['blue','vanilla'])

[[Document(id='83bb873f-4905-41b8-9294-1d119326a7a5', metadata={'author': 'Cihan'}, page_content='I like blue')],
 [Document(id='324478cc-bbca-439f-9d3b-600a6174d5b5', metadata={'author': 'Cihan'}, page_content='I like chocolate ice cream, vanilla ice cream, strawberry ice cream, pistachio ice cream, and many more')]]

In [ ]:
### RAG
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer the following question using the information in the documents:
{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {
    'context': retriever,
    'question': RunnablePassthrough()
} | prompt | model

response = rag_chain.invoke('What is my favorite color?')
response.content

"According to the provided document, your favorite color is blue. The document with the author 'Cihan' states 'I like blue'."